In [ ]:
%matplotlib inline
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

The dataset used in this notebook is the Auto MPG dataset from the UCI Machine Learning Repository: (https://archive.ics.uci.edu/ml/datasets/Auto+MPG).

The mpg column corresponds to the response variable. All the predictor variables can be interpreted as numerical variables except origin, that is categorical.

In [ ]:
df = pd.read_csv('data/auto-mpg.csv', sep='\s+', header=None, usecols=range(8))
df.columns = ['mpg', 'cylinders', 'displacement',
              'horsepower', 'weight', 'acceleration',
              'model_year', 'origin']
df.head()

Removing rows with missing values:

In [ ]:
df = df[~(df.horsepower == '?')]
df.horsepower = df.horsepower.astype(float)

In [ ]:
fig, ax = plt.subplots(2,4)

ax[0,0].hist(df.mpg)
ax[0,0].set_title('mpg distribution')

ax[0,1].hist(df.cylinders)
ax[0,1].set_title('cylinders distribution')

ax[0,2].hist(df.displacement)
ax[0,2].set_title('displacement distribution')

ax[0,3].hist(df.horsepower)
ax[0,3].set_title('horsepower distribution')

ax[1,0].hist(df.weight)
ax[1,0].set_title('weight distribution')

ax[1,1].hist(df.acceleration)
ax[1,1].set_title('acceleration distribution')

ax[1,2].hist(df.model_year)
ax[1,2].set_title('model_year distribution')

ax[1,3].hist(df.origin)
ax[1,3].set_title('origin distribution')

fig.set_figwidth(16)
fig.set_figheight(4)
plt.tight_layout()

In order to use categorical predictor variables in our multiple regression model (in our case only the 'origin' predictor is categorical) we need to transform them into indicator or dummy variables. An indicator variable takes the value 1 or 0 depending on whether the value represented by the indicator variable was the value of the original categorical variable in the original dataset. If the categorical variable only has two levels only one indicator variable is required. Otherwise, we will need to create one indicator variable for each value of the categorical variable.

In [ ]:
print(np.unique(df.origin))

In [ ]:
for v in np.unique(df.origin):
    df['origin_' + str(v)] = (df.origin == v).astype(int)
del df['origin']
df.head()